In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
import math

In [9]:
movies = pd.read_table("movies.dat",sep="::",encoding="ISO-8859-1")
Ratings = pd.read_table("ratings.dat",sep="::")
Tags = pd.read_table("tags.dat",sep="::", encoding="ISO-8859-1")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel 

In [0]:
Ratings.columns = ['userId', 'MovieID', 'rating', 'Timestamp']

In [11]:
Mean = Ratings.groupby(by='userId')['rating'].mean()
print(Mean)

userId
1       5.000000
2       3.200000
3       3.939394
4       4.026316
5       3.850575
          ...   
5342    3.461538
5343    3.093023
5344    3.577143
5345    4.130435
5346    3.346535
Name: rating, Length: 5172, dtype: float64


In [12]:
Mean = Mean.to_frame().reset_index()
print(Mean)

      userId    rating
0          1  5.000000
1          2  3.200000
2          3  3.939394
3          4  4.026316
4          5  3.850575
...      ...       ...
5167    5342  3.461538
5168    5343  3.093023
5169    5344  3.577143
5170    5345  4.130435
5171    5346  3.346535

[5172 rows x 2 columns]


In [13]:
Rating_avg = pd.merge(Ratings,Mean,on='userId')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head(30)

,userId,MovieID,rating_x,Timestamp,rating_y,adg_rating
0,1,185,5.0,838983525,5.0,0.0
1,1,231,5.0,838983392,5.0,0.0
2,1,292,5.0,838983421,5.0,0.0
3,1,316,5.0,838983392,5.0,0.0
4,1,329,5.0,838983392,5.0,0.0
5,1,355,5.0,838984474,5.0,0.0
6,1,356,5.0,838983653,5.0,0.0
7,1,362,5.0,838984885,5.0,0.0
8,1,364,5.0,838983707,5.0,0.0
9,1,370,5.0,838984596,5.0,0.0


In [0]:
movies.columns=['MovieID', 'Title', 'Genres']

In [0]:
Tags.columns=['userId','MovieID', 'Tags', 'Timestamp']

In [16]:
check = pd.pivot_table(Rating_avg,values='rating_x',index='userId', columns='MovieID')
check.head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,62834,62849,62956,62970,62999,63033,63062,63082,63113,63131,63273,63329,63433,63436,63458,63479,63698,63836,63853,63859,63876,63992,64030,64153,64231,64249,64285,64497,64508,64614,64620,64716,64750,64839,64900,64906,64957,64969,64997,65133
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,3.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
final = pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='MovieID')
final.head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,62834,62849,62956,62970,62999,63033,63062,63082,63113,63131,63273,63329,63433,63436,63458,63479,63698,63836,63853,63859,63876,63992,64030,64153,64231,64249,64285,64497,64508,64614,64620,64716,64750,64839,64900,64906,64957,64969,64997,65133
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.026316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.973684,NaN,NaN,NaN,NaN,-1.026316,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-2.850575,NaN,NaN,NaN,NaN,NaN,-0.850575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.850575,NaN,NaN,-0.850575,NaN,1.149425,NaN,1.149425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Replacing NaN by Movie Average
final_movie = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [19]:
final_movie.head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,62834,62849,62956,62970,62999,63033,63062,63082,63113,63131,63273,63329,63433,63436,63458,63479,63698,63836,63853,63859,63876,63992,64030,64153,64231,64249,64285,64497,64508,64614,64620,64716,64750,64839,64900,64906,64957,64969,64997,65133
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.343646,-0.330514,-0.301526,-0.603042,-0.410844,0.315879,-0.174129,-0.309263,-0.38221,-0.04479,0.113213,-0.803881,-0.324209,-0.176954,-0.612045,0.215637,0.386182,-0.178429,-0.870896,-0.644559,0.025310,-0.102946,-0.336056,-0.185398,0.096145,0.143565,-0.07305,0.465899,0.4706,0.128491,-0.266202,0.306545,0.886839,0.105253,-0.001704,0.430834,1.3,-0.571088,-0.110219,0.185338,...,-1.363783,0.57189,0.536994,-0.04745,-0.359537,0.176012,0.21794,0.569792,0.080119,1.127394,-0.797004,-0.687232,0.450046,-0.313945,-1.080394,-0.516245,0.875536,-1.363783,-0.302258,0.021743,0.720124,-0.162129,0.136217,-1.123779,0.91885,-0.286597,-0.363783,-0.602876,-1.363783,0.51269,0.77117,-0.825763,-0.111645,-0.203644,-0.111645,-0.156915,0.234353,-0.462222,0.018223,-0.363783
2,0.343646,-0.330514,-0.301526,-0.603042,-0.410844,0.315879,-0.174129,-0.309263,-0.38221,-0.04479,0.113213,-0.803881,-0.324209,-0.176954,-0.612045,0.215637,0.386182,-0.178429,-0.870896,-0.644559,0.025310,-0.102946,-0.336056,-0.185398,0.096145,0.143565,-0.07305,0.465899,0.4706,0.128491,-0.266202,0.306545,0.886839,0.105253,-0.001704,0.430834,1.3,-0.571088,-0.110219,0.185338,...,-1.363783,0.57189,0.536994,-0.04745,-0.359537,0.176012,0.21794,0.569792,0.080119,1.127394,-0.797004,-0.687232,0.450046,-0.313945,-1.080394,-0.516245,0.875536,-1.363783,-0.302258,0.021743,0.720124,-0.162129,0.136217,-1.123779,0.91885,-0.286597,-0.363783,-0.602876,-1.363783,0.51269,0.77117,-0.825763,-0.111645,-0.203644,-0.111645,-0.156915,0.234353,-0.462222,0.018223,-0.363783
3,0.343646,-0.330514,-0.301526,-0.603042,-0.410844,0.315879,-0.174129,-0.309263,-0.38221,-0.04479,0.113213,-0.803881,-0.324209,-0.176954,-0.612045,0.215637,0.386182,-0.178429,-0.870896,-0.644559,0.025310,-0.102946,-0.336056,-0.185398,0.096145,0.143565,-0.07305,0.465899,0.4706,0.128491,-0.266202,0.306545,0.886839,0.105253,-0.001704,0.430834,1.3,-0.571088,-0.110219,0.185338,...,-1.363783,0.57189,0.536994,-0.04745,-0.359537,0.176012,0.21794,0.569792,0.080119,1.127394,-0.797004,-0.687232,0.450046,-0.313945,-1.080394,-0.516245,0.875536,-1.363783,-0.302258,0.021743,0.720124,-0.162129,0.136217,-1.123779,0.91885,-0.286597,-0.363783,-0.602876,-1.363783,0.51269,0.77117,-0.825763,-0.111645,-0.203644,-0.111645,-0.156915,0.234353,-0.462222,0.018223,-0.363783
4,0.343646,-0.330514,-0.301526,-0.603042,-0.410844,0.315879,-0.174129,-0.309263,-0.38221,-0.04479,0.113213,-0.803881,-0.324209,-0.176954,-0.612045,0.215637,0.386182,-0.178429,-0.870896,-0.644559,-1.026316,-0.102946,-0.336056,-0.185398,0.096145,0.143565,-0.07305,0.465899,0.4706,0.128491,-0.266202,0.306545,0.886839,0.973684,-0.001704,0.430834,1.3,-0.571088,-1.026316,0.185338,...,-1.363783,0.57189,0.536994,-0.04745,-0.359537,0.176012,0.21794,0.569792,0.080119,1.127394,-0.797004,-0.687232,0.450046,-0.313945,-1.080394,-0.516245,0.875536,-1.363783,-0.302258,0.021743,0.720124,-0.162129,0.136217,-1.123779,0.91885,-0.286597,-0.363783,-0.602876,-1.363783,0.51269,0.77117,-0.825763,-0.111645,-0.203644,-0.111645,-0.156915,0.234353,-0.462222,0.018223,-0.363783
5,-2.850575,-0.330514,-0.301526,-0.603042,-0.410844,0.315879,-0.850575,-0.309263,-0.38221,-0.04479,0.113213,-0.803881,-0.324209,-0.176954,-0.612045,0.215637,0.386182,-0.178429,-0.870896,-0.644559,0.025310,-0.102946,-0.336056,-0.185398,-0.850575,0.143565,-0.07305,-0.850575,0.4706,1.149425,-0.266202,1.149425,0.886839,0.105253,-0.001704,0.430834,1.3,-0.571088,-0.110219,0.185338,...,-1.363783,0.57189,0.536994,-0.04745,-0.359537,0.176012,0.21794,0.569792,0.080119,1.127394,-0.797004,-0.687232,0.450046,-0.313945,-1.080394,-0.

In [20]:
final_user.head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,62834,62849,62956,62970,62999,63033,63062,63082,63113,63131,63273,63329,63433,63436,63458,63479,63698,63836,63853,63859,63876,63992,64030,64153,64231,64249,64285,64497,64508,64614,64620,64716,64750,64839,64900,64906,64957,64969,64997,65133
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,...,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16,-1.776357e-16
3,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,...,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17,-5.382900e-17
4,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.402387e-16,1.

In [22]:
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,17,18,19,22,23,24,26,27,28,29,30,33,34,35,36,37,38,40,41,42,43,44,45,46,47,...,5306,5307,5308,5309,5310,5311,5312,5313,5314,5315,5316,5317,5318,5319,5320,5321,5322,5323,5324,5326,5327,5328,5329,5330,5331,5332,5333,5334,5335,5336,5337,5338,5339,5340,5341,5342,5343,5344,5345,5346
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000
2,0.0,0.000000,6.433737e-03,0.128475,6.565701e-02,2.313779e-02,-6.229032e-02,-0.053086,-6.162976e-33,0.012136,0.039019,-2.584121e-02,0.019453,7.031308e-02,1.396203e-01,-0.017806,-0.003735,0.106295,0.110546,7.965318e-02,1.490281e-01,-0.000587,-4.010148e-02,-5.950056e-30,-3.953036e-30,0.069684,4.438418e-02,0.104738,0.003906,-0.005650,0.050781,-0.006150,9.771336e-30,0.003334,0.041519,-4.073043e-03,-9.602173e-30,-0.015138,-0.007975,0.092649,...,-5.189405e-30,-0.015571,-5.650057e-02,-0.187929,-6.818842e-02,-1.123858e-01,0.220767,0.005379,0.023778,0.066177,-0.067735,2.194910e-18,9.653588e-02,-1.541488e-29,2.194858e-02,0.053928,4.259237e-02,-0.109309,-1.786601e-02,2.260324e-01,0.037402,-0.030313,0.045165,-0.055554,-2.021853e-03,-5.392604e-33,0.077300,0.060225,0.081702,1.657525e-29,0.003678,-2.311116e-33,0.133335,0.146893,-7.170090e-30,0.045043,0.142066,0.065971,-1.356460e-29,0.014468
3,0.0,0.006434,0.000000e+00,0.003989,-3.885624e-30,2.536762e-30,-8.664097e-03,0.063458,-3.627965e-02,0.014281,0.085189,2.390545e-30,0.009693,-5.319280e-03,-2.369212e-30,-0.010365,0.024155,0.008637,0.033182,-3.849241e-30,9.484624e-30,0.043597,-2.497375e-30,-1.861379e-30,-1.099007e-30,0.123969,2.165689e-30,0.018044,0.046783,-0.007939,0.010297,-0.008860,2.470946e-30,-0.015451,0.008157,4.915567e-30,-5.108448e-03,0.020828,-0.050861,-0.007043,...,-1.185613e-30,-0.012539,-3.166866e-03,0.042299,5.130231e-30,3.960390e-30,0.006246,0.006691,-0.002929,-0.000220,0.008454,-2.507911e-30,-1.201179e-30,-4.374826e-30,-2.313950e-30,0.005116,2.369411e-30,0.022005,-2.572648e-31,-1.340867e-30,0.027782,-0.004406,0.028767,-0.006620,-5.202421e-31,3.200486e-03,0.036174,-0.006049,0.016824,4.620520e-30,-0.002016,1.431088e-03,0.026706,0.018608,-3.788953e-03,-0.010138,0.004410,0.010021,-4.110922e-30,-0.010781
4,0.0,0.128475,3.988538e-03,0.000000,6.493263e-04,-1.469944e-03,-1.452921e-31,0.008249,-7.703720e-34,-0.012302,-0.038646,1.155885e-02,-0.066053,7.220103e-30,9.705323e-04,-0.012879,-0.020819,0.011205,0.148795,6.781069e-02,-1.442020e-29,0.109218,4.006631e-30,2.578074e-30,1.490506e-30,0.064385,-3.148014e-30,0.069989,0.054484,0.010455,0.022083,0.015631,-1.317386e-02,-0.005685,-0.120907,-2.761770e-02,-1.925757e-02,0.047716,0.008412,-0.000236,...,2.119371e-30,0.080679,-4.252905e-02,-0.162956,-9.438168e-30,-5.700585e-30,0.071341,0.004863,0.011779,0.035632,0.053394,-2.510518e-02,1.932605e-30,6.382826e-30,-2.879697e-02,0.018539,9.401245e-02,0.004205,4.031587e-31,2.276181e-30,-0.011098,-0.014082,0.099654,0.061487,-1.878557e-02,-7.703720e-34,0.109773,0.030740,0.070175,1.814250e-02,0.035852,1.501808e-03,0.063511,0.190179,2.809211e-30,0.012840,0.149119,0.017085,2.642891e-02,0.020841
5,0.0,0.065657,-3.885624e-30,0.000649,0.000000e+00,1.982153e-02,3.1

In [23]:
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,17,18,19,22,23,24,26,27,28,29,30,33,34,35,36,37,38,40,41,42,43,44,45,46,47,...,5306,5307,5308,5309,5310,5311,5312,5313,5314,5315,5316,5317,5318,5319,5320,5321,5322,5323,5324,5326,5327,5328,5329,5330,5331,5332,5333,5334,5335,5336,5337,5338,5339,5340,5341,5342,5343,5344,5345,5346
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.997886,0.996897,0.993918,0.988739,0.996095,0.990378,0.943656,0.996049,0.989449,0.995135,0.996719,0.985674,0.986565,0.996421,0.991154,0.978984,0.977003,0.995679,0.996911,0.996956,0.997162,0.996668,0.997324,0.985853,0.989407,0.998525,0.931134,0.983988,0.957050,0.984059,0.967892,0.992535,0.985330,0.997756,0.983401,0.992657,0.991206,0.995321,0.990596,...,0.994253,0.988836,0.993507,0.998727,0.997887,0.997776,0.989156,0.991254,0.969025,0.995464,0.987964,0.992997,0.991725,0.997484,0.994012,0.985417,0.997707,0.997319,0.996522,0.998764,0.987652,0.911881,0.966192,0.985072,0.994442,0.996543,0.993994,0.986495,0.992740,0.993766,0.991405,0.996820,0.985452,0.984560,0.996150,0.996368,0.981948,0.968540,0.991271,0.964546
2,0.997886,0.000000,0.995378,0.993459,0.987722,0.994823,0.988529,0.941501,0.994789,0.987981,0.993981,0.995569,0.984581,0.985612,0.995387,0.989435,0.977545,0.977193,0.994876,0.995649,0.996000,0.995608,0.995094,0.996063,0.984610,0.988781,0.997284,0.931842,0.982750,0.955009,0.983438,0.966663,0.991171,0.984327,0.996382,0.981962,0.991104,0.989808,0.994273,0.989658,...,0.992995,0.987845,0.991711,0.996801,0.996457,0.995951,0.989095,0.990168,0.967765,0.994430,0.985815,0.991706,0.990491,0.996222,0.992713,0.984554,0.996702,0.995703,0.995173,0.997743,0.986407,0.909656,0.965528,0.983921,0.993172,0.995283,0.992608,0.985558,0.991479,0.992565,0.990087,0.995560,0.984691,0.984223,0.994890,0.995236,0.981560,0.967916,0.990078,0.963360
3,0.996897,0.995378,0.000000,0.991855,0.986512,0.993852,0.988250,0.943084,0.993608,0.987225,0.993301,0.994474,0.983696,0.984165,0.994159,0.988602,0.977074,0.974927,0.993563,0.994637,0.994710,0.995114,0.994423,0.995077,0.983631,0.988938,0.996276,0.928820,0.982485,0.954896,0.982615,0.966306,0.990190,0.983087,0.995485,0.981041,0.990404,0.989164,0.992960,0.988187,...,0.992013,0.986637,0.991322,0.996648,0.995639,0.995528,0.986932,0.988918,0.966935,0.993399,0.985754,0.990761,0.989491,0.995237,0.991773,0.983283,0.995603,0.995320,0.994278,0.996514,0.985216,0.909438,0.965057,0.983487,0.992202,0.994591,0.991602,0.984290,0.990179,0.991584,0.988763,0.994558,0.983119,0.982447,0.993875,0.994032,0.979158,0.966564,0.989099,0.962006
4,0.993918,0.993459,0.991855,0.000000,0.983855,0.991194,0.985481,0.938422,0.991124,0.984160,0.989769,0.991886,0.979385,0.981685,0.991370,0.985949,0.972303,0.971752,0.991767,0.992369,0.992027,0.992301,0.991741,0.992393,0.980984,0.985039,0.993587,0.928091,0.979821,0.952905,0.979849,0.963178,0.987330,0.980405,0.991801,0.978177,0.987215,0.986847,0.990685,0.984948,...,0.989337,0.984449,0.988067,0.992167,0.992954,0.992843,0.985018,0.986333,0.964756,0.990990,0.983283,0.987739,0.986821,0.992552,0.988648,0.980728,0.993464,0.992169,0.991594,0.993826,0.981459,0.906425,0.963024,0.981028,0.989253,0.991617,0.988940,0.982025,0.988063,0.989186,0.986673,0.991848,0.980942,0.982574,0.991225,0.991341,0.978449,0.963913,0.986696,0.960289
5,0.988739,0.987722,0.986512,0.983855,0.000000,0.985832,0.980305,0.932979,0.985129,0.981837,0.984943,0.983880,0.976301,0.976781,0.986088,0.980358,0.968300,0.967166,0.985477,0.987148,0.985387,0.986199,0.984300,0.986888,0.975586,0.978246,0.988608,0.919268,0.974525,0.947009,0.974513,0.958590,0.982080,0.976470,0.987532,0.972453,0.982239,0.981051,0.985168,0.981965,...,0.983895,0.978286,0.983094,0.988448,0.987284,0.987584,0.978510,0.980444,0.958843,0.985525,0.977180,0.983053,0.982631,0.987053,0.983760,0.974451,0.987949,0.985222,0.985271,0.988184,0.977559,0.904100,0.957827,0.975400,0.983287,0.986249,0.983539,0.975331,0.980238,0.983596,0.981641,0.986270,0.975271,0.972566,0.985786,0.985

In [0]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [25]:
# top 30 neighbours for each user
sim_user_30_u = find_n_neighbours(similarity_with_user,30)
sim_user_30_u.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5346,1779,1801,1802,1803,1804,1805,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1800,1799,1798,1788,1781,1782,1783,1784,1785,1786,1787
2,4423,2519,3000,3566,4046,3328,4922,3262,2343,2881,1779,1648,3933,4055,3018,1825,4519,4961,3677,1975,766,5144,84,1340,3051,2371,2183,4202,5083,752
3,1524,4934,1342,4660,4786,2421,4387,3521,818,3105,3765,30,1354,4828,3641,117,2196,3896,2440,5029,3986,2122,2136,583,4743,763,3085,3264,2724,1578
4,758,4644,2694,4709,4755,4131,5086,2565,789,609,4205,1243,54,3566,4179,3024,2173,4477,3945,1706,2493,1511,2207,3231,2549,4079,199,1927,3744,2263
5,2089,4068,866,796,3644,4288,2903,337,4507,575,2550,1164,1549,2372,4967,411,3316,3596,1421,112,4853,229,3625,2155,167,2363,5300,1394,1286,98


In [26]:
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4771,1462,445,1260,3918,478,4860,2274,4082,3854,1096,1600,2236,2719,1779,2880,71,4768,548,176,3790,1975,1615,1375,4392,2362,3141,2803,3457,4300
2,2519,4392,71,1779,2274,1975,3262,1,4202,4046,2289,3000,3018,2473,2737,4771,5326,4922,172,5144,2597,3854,1044,1260,4860,311,4788,1600,3431,4082
3,4771,1,445,2880,1524,4082,1462,478,3790,2274,1096,176,4860,71,3918,3854,1600,2719,5309,1975,1779,4768,548,1375,1615,751,1889,2362,1964,3363
4,2694,4709,4644,4755,758,4131,2529,176,3341,2565,71,5292,3431,4392,4179,4891,3566,2473,2609,4775,3854,2830,4205,5086,4477,1975,1501,3262,416,641
5,796,4967,1421,3644,3316,2372,4961,4267,4922,3238,1821,5195,3269,4507,1626,337,575,4771,4684,411,2274,84,1416,462,1462,2529,4764,2719,95,2358


In [0]:
def get_user_similar_movies( user1, user2 ):
    common_movies = Rating_avg[Rating_avg.userId == user1].merge(
    Rating_avg[Rating_avg.userId == user2],
    on = 'MovieID',
    how = "inner" )
    return common_movies.merge( movies, on = 'MovieID' )

In [0]:
a = get_user_similar_movies(370,86309)
a = a.loc[ : , ['rating_x_x','rating_x_y','Title']]

In [0]:
def User_item_score(user,item):
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = final_movie.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_movie.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [30]:
score = User_item_score(320,7371)
print(score)

4.234452659959687


In [0]:
Rating_avg = Rating_avg.astype({"MovieID": str})
Movie_user = Rating_avg.groupby(by = 'userId')['MovieID'].apply(lambda x:','.join(x))


In [0]:
def User_item_score1(user):
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_movie.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_movie.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'MovieID':Movies_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    Movie_Name = top_5_recommendation.merge(movies, how='inner', on='MovieID')
    Movie_Names = Movie_Name.Title.values.tolist()
    return Movie_Names

In [35]:
user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for User Id : 370")
print("   ")
for i in predicted_movies:
    print(i)

Enter the user id to whom you want to recommend : 370
 
The Recommendations for User Id : 370
   
Shawshank Redemption, The (1994)
Usual Suspects, The (1995)
Godfather, The (1972)
Schindler's List (1993)
Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le) (2001)
